# Opencast Mining

## Objective and Prerequisites

How can a mining company use mathematical optimization to identify which excavation locations to choose in order to maximize the gross margins of extracting ore? Try this modeling example to find out!

This model is example 14 from the fifth edition of Model Building in Mathematical Programming by H. Paul Williams on pages 269-270 and 324-325.

This example is at the intermediate level where we assume that you know Python and the Gurobi Python API and that you have some knowledge of building mathematical optimization models.

**Download the Repository** <br /> 
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip). 

## Problem Description

A company has obtained permission to conduct opencast mining within a square plot 200 ft $\times$ 200 ft. The angle of slip of the soil is such that it is not possible for the sides of the excavation to be steeper than 45 degrees. The company has obtained estimates for the value of the ore in various places at various depths. Bearing in mind the restrictions imposed by the angle of slip, the company decides to consider the problem as one of the extracting of rectangular blocks. Each block has horizontal dimensions 50 ft $\times$ 50 ft and a vertical dimension of 25 ft. If the blocks are chosen to lie above one another then it is only possible to excavate blocks forming an upturned pyramid. 
The three dimensional representation below shows four levels of excavation. We have numbered, in black, each block at each level, and the number in red represents the block underneath the four blocks of the level. For example, block 17 of level 2 lies underneath the blocks 1,2,5,and 6 of level 1.
![pyramid](extractionPyramid.PNG)

The profit for the extraction of ore at each block has been estimated. The goal is to find an ore extraction plan that maximizes total profit.

## Model Formulation

### Sets and Indices

$b,b2 \in \text{Blocks}=\{1,...,30 \}$.

### Parameters

$\text{profit}_{b} \in \mathbb{R}^+$: Profit from extracting ore from block $b$.

$(b,b2) \in Arcs = Blocks \times Blocks$: This parameter represent the arcs in the series-parallel graph describing the rules of extraction. The arc $(b,b2)$ in the adjacency matrix of this series-parallel graph has a value of 1 if block b2 is one of the four blocks above block b, and 0 otherwise. For example, arc $(29,24)$ represents that block 24 is one of the four blocks above block 29.

### Decision Variables

$\text{extract}_{b} \in \{0,1\}$: This binary variable is equal 1, if block $b$ is selected, and 0 otherwise.

### Constraints

**Extraction**: If a block is extracted, then the four blocks above it must also be extracted..

\begin{equation}
\text{extract}_{b2} \geq \text{extract}_{b} \quad \forall (b,b2) \in \text{Arcs}
\end{equation}

### Objective Function

**Profits**: Maximize profits from the extraction of ore.

\begin{equation}
\text{Maximize} \quad \sum_{b \in Blocks} \text{profit}_{b}*\text{extract}_{b}
\end{equation}

## Python Implementation

We import the Gurobi Python Module.

In [ ]:
%pip install gurobipy

In [1]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.11 & Gurobi 11.0

## Input data

We define all the input data for the model and other Python libraries.

In [2]:
# Create a dictionary to capture the profit of the extracting of ore at each block.

blocks, profit = gp.multidict({
    ('1'): 0,
    ('2'): 0,
    ('3'): 0,
    ('4'): -1500,
    ('5'): 0,
    ('6'): 1000,
    ('7'): 0,
    ('8'): -1500,
    ('9'): -1000,
    ('10'): -1000,
    ('11'): -1500,
    ('12'): -2000,
    ('13'): -1500,
    ('14'): -1500,
    ('15'): -2000,
    ('16'): -2500,
    ('17'): 2000,
    ('18'): 2000,
    ('19'): -2000,
    ('20'): 0,
    ('21'): 0,
    ('22'): -4000,
    ('23'): -2000,
    ('24'): -2000,
    ('25'): -5000,
    ('26'): 16000,
    ('27'): 4000,
    ('28'): 2000,
    ('29'): 0,
    ('30'): 2000
})

# Create a dictionary for the adjacency matrix of the series-parallel graph.

arcs, value = gp.multidict({
    ('30','26'): 1,
    ('30','27'): 1,
    ('30','28'): 1,
    ('30','29'): 1,
    ('29','21'): 1,
    ('29','22'): 1,
    ('29','24'): 1,
    ('29','25'): 1,
    ('28','20'): 1,
    ('28','21'): 1,
    ('28','23'): 1,
    ('28','24'): 1,
    ('27','18'): 1,
    ('27','19'): 1,
    ('27','21'): 1,
    ('27','22'): 1,
    ('26','17'): 1,
    ('26','18'): 1,
    ('26','20'): 1,
    ('26','21'): 1,
    ('25','11'): 1,
    ('25','12'): 1,
    ('25','15'): 1,
    ('25','16'): 1,
    ('24','10'): 1,
    ('24','11'): 1,
    ('24','14'): 1,
    ('24','15'): 1,
    ('23','9'): 1,
    ('23','10'): 1,
    ('23','13'): 1,
    ('23','14'): 1,
    ('22','7'): 1,
    ('22','8'): 1,
    ('22','11'): 1,
    ('22','12'): 1,
    ('21','6'): 1,
    ('21','7'): 1,
    ('21','10'): 1,
    ('21','11'): 1,
    ('20','5'): 1,
    ('20','6'): 1,
    ('20','9'): 1,
    ('20','10'): 1,
    ('19','3'): 1,
    ('19','4'): 1,
    ('19','7'): 1,
    ('19','8'): 1,
    ('18','2'): 1,
    ('18','3'): 1,
    ('18','6'): 1,
    ('18','7'): 1,
    ('17','1'): 1,
    ('17','2'): 1,
    ('17','5'): 1,
    ('17','6'): 1
})

## Model Deployment

We create a model and the variables. These binary decision variables define which block to extract ore from.

Notice that the matrix of coefficients of the constraints is totally unimodular, therefore the decision variables can be defined in the interval $[0,1]$ and the problem can be solved as a linear programming problem.

In [3]:
model = gp.Model('opencastMining')

# Decision variable to extract ore from block
extract = model.addVars(blocks, ub=1, vtype=GRB.CONTINUOUS, name="extract" )

Using license file c:\gurobi\gurobi.lic


The following constraints ensure that if a block is extracted, then the four blocks above it must also be extracted.

In [4]:
# Extraction constraints

extractionConstrs = model.addConstrs( (extract[b] <= extract[b2]  for b,b2 in arcs), name='extractionConstrs')

We want to maximize the profits from the extraction of ore.

In [5]:
# Objective function

extractionProfit = gp.quicksum(profit[b]*extract[b] for b in blocks )

model.setObjective(extractionProfit, GRB.MAXIMIZE)

In [6]:
# Verify model formulation

model.write('opencastMining.lp')

# Run optimization engine

model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 56 rows, 30 columns and 112 nonzeros
Model fingerprint: 0x83427493
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+03, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 22 rows and 12 columns
Presolve time: 0.01s
Presolved: 34 rows, 18 columns, 68 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.3014500e+04   1.100300e+01   0.000000e+00      0s
       9    1.7500000e+04   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.01 seconds
Optimal objective  1.750000000e+04


## Analysis

The total profit generated from the optimal ore extraction plan is $\$17,500.00$. 
The block to extract ore from and its associated profit or loss are shown in the following table.

In [7]:
# Output reports

extraction_plan = pd.DataFrame(
    [(b, f"${profit[b]*round(extract[b].x):,.2f}") for b in blocks if (extract[b].x > 0.5)],
    columns = ["Block", "Profit/Loss"],
)
extraction_plan.index=[''] * len(extraction_plan)
extraction_plan

,Block,Profit/Loss
,1,$0.00
,2,$0.00
,3,$0.00
,5,$0.00
,6,"$1,000.00"
,7,$0.00
,9,"$-1,000.00"
,10,"$-1,000.00"
,11,"$-1,500.00"
,17,"$2,000.00"


## References

H. Paul Williams, Model Building in Mathematical Programming, fifth edition.

Copyright © 2020 Gurobi Optimization, LLC